In [1]:
from datetime import datetime as dt, timedelta
start_date = dt(2016, 1, 30, 0, 0, 0) #best results when train from 4/2016 and onward # maybe do small window retraining
end_date = dt(2016, 11, 23, 0, 0, 0)
price_window = 350
tweet_window = 250
min_tweets = 1
cutoff = .25
retweets_min = 0

In [2]:
import pandas as pd
df = pd.read_csv('data_new.csv', parse_dates=['Date','Date_Time'])
start_index = df[df['Date'] == start_date].iloc[[0]].index.values[0]
df = df[start_index:]
df = df.set_index(['Date_Time'])
df = df[start_date:]
# print(df)

In [3]:
import numpy as np
from collections import Counter

def getNearest(t, df):
    i = df.index.searchsorted(t)
    return df.iloc[i]['Price'] #df.ix[df.index[i]]['Price']

def getDelta(df, date, delta_minutes, cutoff):
    df2 = df[:-(2*delta_minutes)]
    date = np.datetime64(date)
    delta_minutes = np.timedelta64(delta_minutes,'m')
    m = getNearest(date + delta_minutes, df2) - getNearest(date, df2)
    if m > cutoff: return 2
    if m < -cutoff: return 0
    return 1 

In [4]:
#get percent of increases 
df2 = df[:-(5*price_window)]
number_samples = int(len(df2)/25)
dlist = np.random.choice(df2.index.values, number_samples)
dlist = [getDelta(df, d, price_window, 0) for d in dlist]
dlist_count = Counter(dlist)
nonTweet_percentPos = dlist_count[2]/(number_samples - dlist_count[1])
print('For', number_samples, 'instances of', price_window, 'minute periods, The percent pos is:', nonTweet_percentPos)   

For 17139 instances of 350 minute periods, The percent pos is: 0.5329949238578681


In [5]:
words_i_like = ['bitcoin', 'btc', 'blockchain', 'litecoin', 'usd',
'wallet', 'currency', 'altcoin', 'mining', 'gox', 'mt', 'crypto', 'new', 
'cryptocurrency', 'ethereum', 'fintech', 'ltc', 'free', 'digital', 'latest',
'money', 'bank', 'hardware', 'index', 'satoshi', 'market', 'economy', 'bitcoins',
'dogecoin', 'value', 'secure', 'miner', 'trading', 'coindesk', 'smart', 
'bitstamp', 'technology', 'euro', 'buy', 'trade', 
'coinbase', 'power', 'time', 'tech', 'trezor', 'bitfinex', 'algorithm', 'china', 'banks', 
'earn', 'past', 'landbitcoin', 'portal', 'win',
 'data', 'coin', 'best', 'cash', 'bitcoinnews', 'increased', 'cloud', 'average', 
'future', 'change', 'financial', 'virtual', 'startup', 'open', 'ceo', 'platform', 
'decreased', 'business', 'finance', 'convert', 'high', 'dash', 'altcoins', 'currencies', 
'collapse', 'libertarian', 'bot', 'dollar', 'movement', 'directly', 'game',
'global', 'technical', 'investment', 'launches', 'volume', 'network', 'support',
'observer', 'lost', 'security', 'secure', 'won', 'good', 'launch', 
'gambling', 'japan', 'invest', 'sell', 'wild', 'hack', 
'pay', 'exchanges', 'miners', 'crypto-currencies', 'forum', 'fast', 'sell', 'ledger', 
'mobile', 'grow', 'hot', 'great', 'wild', 'hack', 'miracle', 'bullish', 'solution', 'millionare']

In [6]:
from collections import defaultdict
from textblob import TextBlob

#TODO: try grabbing each minute seperately

def singleTweetFV(tweet_text, retweets):
    words = [w.lstrip('#') for w in tweet_text.lower().split(' ')]
    counts = Counter(words)
    fv = np.zeros(len(words_i_like))
    for i, word in enumerate(words_i_like): fv[i] = counts[word]
    fv *= (retweets + 1)
    return fv

def sentimentTweet(tweet_text):
    b = TextBlob(tweet_text)
    fv = np.array([b.sentiment.subjectivity, 1 + b.sentiment.polarity])
    return fv

print(sentimentTweet("I hate this"))

[ 0.9  0.2]


In [7]:
# from pymongo import MongoClient
# twitter_start_date = start_date + timedelta(days=1)

# db = MongoClient("mongodb://104.236.1.250:27017")['local']
# collection = db['twitter_two']
# print('regular grabbing tweets')
# tweets = list(collection.find({"date": {"$gt": twitter_start_date, "$lt": end_date}, "retweets": {"$gte":retweets_min}}))
# print('# of tweets grabbed:', len(tweets))
# print(tweets[1:10])

# def buildFVs(df, tweets, price_window, tweet_window, min_tweets, cutoff):
#     mFV = np.zeros(len(words_i_like))
#     date = tweets[0]['date']
#     last_date = df['Date'][-1] - timedelta(days=1)
#     tweet_count = 0
#     fvs = []
#     targetdata =[]
#     most_recent = []
#     counts = []
    
#     for tweet in tweets:
#         tFV = singleTweetFV(tweet['text'], tweet['retweets'])
#         if date.minute - tweet['date'].minute == 0:
#             mFV += tFV
#             tweet_count +=1
#         else:
#             if tweet_count > min_tweets:
#                 if date > last_date: break # return fvs, targetdata
#                 most_recent.append(mFV)
#                 if len(most_recent) > tweet_window:
#                     most_recent.pop(0) 
#                     fvs.append(sum(most_recent))
#                     targetdata.append(getDelta(df, np.datetime64(date), price_window, cutoff))
#                     counts.append(tweet_count)
#             mFV = np.zeros(len(words_i_like))
#             date = tweet['date']
#             tweet_count = 0
#     return fvs, targetdata

# fvs, target_data = buildFVs(df, tweets, price_window, tweet_window, min_tweets, cutoff)

# print('lenght of target data:', len(target_data))

In [15]:
# set up db
from pymongo import MongoClient

db = MongoClient("mongodb://104.236.1.250:27017")['bitcoin_scraped']
db.authenticate('meir', 'PreemPalver', source='bitcoin_scraped')
collection = db['scraped1']
tweets = list(collection.find({}))
print(tweets)
# collection = db['twitter_two_modified_agg']

[{'_id': ObjectId('58719e2a220fec32f1a79a7c'), 'myid': 'hi'}]


In [11]:
def scrape_window(start_date, end_date):
    print('scraping: ', start_date, end_date)
    twitter_start_date = start_date + timedelta(days=1)
    tweets = list(collection.find({"_id": {"$gt": twitter_start_date, "$lt": end_date}}).sort([("_id", 1)]))
    print(len(tweets))
#     print('length::', len(tweets))
#     for i in tweets[0:20]:
#         print(i)
    return tweets
    
def monthRange(start_date, end_date):
    week = timedelta(weeks=1)
    for n in range(int ((end_date - start_date)/week))[::4]:

        yield start_date + timedelta(weeks=n)

def buildfvs(start_date, last_date):
    fvs = []
    targetdata =[]
    most_recent = []
    counter = 0

    for month in monthRange(start_date, last_date):
        monthFV = []
        monthTarget = []
        for tweet_min in scrape_window(month, month + timedelta(weeks=4)):
            count = tweet_min['count']
            date = tweet_min['_id']
            if count  < min_tweets: continue
            mFV = np.array([count, tweet_min['retweets_total']])
            for tweet_text, retweets in zip(tweet_min['text'], tweet_min['retweets']):
                if retweets >= retweets_min:
                    sent = np.zeros(2)   
                    sent += sentimentTweet(tweet_text) # fix sentiment, seems too negative
                    words = np.zeros(len(words_i_like))  
                    words += singleTweetFV(tweet_text, retweets)
#             print(sent/count)
#             mFV = np.concatenate((sent/count, mFV, words))
            mFV = words
            most_recent.append((mFV, date))
#             counter +=1
#             first = most_recent[0][1]
#             print(len(most_recent))
#             while date - first > timedelta(minutes=tweet_window):
#                 print(date, first, date - first)
#                 print(timedelta(minutes=tweet_window))
#                 most_recent.pop(0) 
#                 first = most_recent[0][1]
            l = len(most_recent) 
            if l > tweet_window:
                most_recent.pop(0)
                sum_month = sum([x[0] for x in most_recent])
#                 sum_month[0] = sum_month[0]/l
#                 sum_month[1] = sum_month[1]/l
                monthFV.append(sum_month)
                monthTarget.append(getDelta(df, np.datetime64(date), price_window, cutoff))
        fvs.append(monthFV)
        targetdata.append(monthTarget)

    return (fvs, targetdata)


# start_date = dt(2014, 1, 1, 0, 0, 0)
last_date = dt(2016, 10, 23, 0, 0, 0) - timedelta(days=1)

fvs, targetdata = buildfvs(start_date, last_date)
print(len(fvs))
print(len(targetdata))

# %lprun -f buildfvs buildfvs(start_date, last_date)
# %lprun -f getDelta getDelta(df, np.datetime64(date), price_window, cutoff)


scraping:  2016-01-30 00:00:00 2016-02-27 00:00:00
0
scraping:  2016-02-27 00:00:00 2016-03-26 00:00:00
0
scraping:  2016-03-26 00:00:00 2016-04-23 00:00:00
0
scraping:  2016-04-23 00:00:00 2016-05-21 00:00:00
0
scraping:  2016-05-21 00:00:00 2016-06-18 00:00:00
0
scraping:  2016-06-18 00:00:00 2016-07-16 00:00:00
0
scraping:  2016-07-16 00:00:00 2016-08-13 00:00:00
0
scraping:  2016-08-13 00:00:00 2016-09-10 00:00:00
0
scraping:  2016-09-10 00:00:00 2016-10-08 00:00:00
0
scraping:  2016-10-08 00:00:00 2016-11-05 00:00:00
0
10
10


In [12]:
def split_list(a_list):
    third = int(len(a_list)/3)
    return a_list[:third], a_list[third:2*third], a_list[2*third:],

fvs_new = [item for sublist in fvs for item in sublist]
targetdata_new = [item for sublist in targetdata for item in sublist]
fva, fvb, fvc = split_list(fvs_new)
tda, tdb, tdc= split_list(targetdata_new)

target_up = [(sum(tda)/len(tdb))/2, (sum(tdb)/len(tdb))/2, (sum(tdc)/len(tdb))/2]
large_target_up = (sum(targetdata_new)/len(targetdata_new))/2

print(len(fva))
print(fva[:5])
print(tda[:50])

ZeroDivisionError: division by zero

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

modeltype = LogisticRegression()

guessedup = 0
clf = modeltype.fit(fva, tda)
pred = clf.predict(fvb)
print(len(pred))

count = total = 0
for i, x in enumerate(pred):
    if x != 1 and tdb[i] != 1:
        count += 1
        if x == 2: guessedup+=1
        if x == tdb[i]: total += 1
            
log_reg = ["Logistic Regression", pred.mean()/2,  total/count, (len(pred) - count)/len(pred)]            

modeltype = MultinomialNB()

guessedup = 0
clf = modeltype.fit(fva, tda)
pred = clf.predict(fvb)

count = total = 0
for i, x in enumerate(pred):
    if x != 1 and tdb[i] != 1:
        count += 1
        if x == 2: guessedup+=1
        if x == tdb[i]: total += 1
if count == 0: count = 1
# print(len(pred))
mnbayes = ["MultinomialNB", pred.mean()/2,  total/count, (len(pred) - count)/len(pred)]            

modeltype = RandomForestClassifier(n_estimators=500)

guessedup = 0
clf = modeltype.fit(fva, tda)
pred = clf.predict(fvb)

count = total = 0
for i, x in enumerate(pred):
    if x != 1 and tdb[i] != 1:
        count += 1
        if x == 2: guessedup+=1
        if x == tdb[i]: total += 1

randtrees = ["RandomForestClassifier", pred.mean()/2,  total/count, (len(pred) - count)/len(pred)]       

print('======== Tuning ========')
print('start date: ', start_date)
print('price window =  minutes after tweet: ', price_window) # '. minutes before:', MINUTES_BEFORE)
print('tweet window = number of tweets looked at: ', tweet_window)
print('minimum tweets in minute: ', min_tweets)
print('minimum retweets: ', retweets_min)
print('cutoff:', cutoff)
print('tweet_accuracy', large_target_up)
print('========================')

print("percent of bitcoin increases over time", nonTweet_percentPos)
print("percentage of target up:", target_up)

for c in [log_reg, mnbayes, randtrees]:
    print('\n')
    print(c[0])
    print("percentage of predictions up:", c[1])
    print('percent_accurate:', c[2])
    print('(amount removed:', str(c[3]) + '%)')